In [1]:
# imports
import openai
from dotenv import load_dotenv
import os

In [2]:
# load env var
load_dotenv()

True

In [3]:
# openai setup
openai.api_key = os.getenv("OPENAI_API_KEY")
model = 'gpt-3.5-turbo'

Observations on the way
- omg the response time is actually quite slow and even then variable. At least a few seconds is expected. Might need to have some indication that output is still being generated instead of hanging there without any feedback

## Ingredient recipe reco prompt test

In [4]:
# chat completions settings

temperature = 0.8 # pretty random response is fine, even preferable not to give same old boring results
user_input_prefix_message = "I want recipe recommendation that includes the following ingredients: "
sample_user_ingredient_input = "black olives, sun-dried tomato"
messages = [
    {"role": "system", "content": "You are a recipe recommendation engine. Given a list of ingredients given by user, give at most 5 recipe that involves said ingredients. Return the name of the recipe, type of cuisine, and a brief description of the dish. If the dish is from a particular cuisine, also include the original language's name in English."},
    {"role": "user", "content": user_input_prefix_message + sample_user_ingredient_input}
]


In [5]:
chat_completion = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    temperature=temperature
)

In [14]:
response_text = chat_completion['choices'][0]['message']['content']
print(response_text)

Here are 5 recipe recommendations that include black olives and sun-dried tomato:

1. Mediterranean Pasta Salad - Mediterranean cuisine (originally called Insalata di pasta mediterranea in Italian): This dish is a cold pasta dish that involves cooking some pasta and mixing it with sun-dried tomatoes, black olives, red onion, feta cheese, and a homemade dressing made with olive oil, red wine vinegar, and dried oregano. It is perfect for a summer day picnic.

2. Sun-Dried Tomato and Olive Tapenade - French cuisine: This is a classic French spread that involves blending sun-dried tomatoes, black olives, capers, garlic, lemon juice, and olive oil, and serving it with toasted bread or crackers as an appetizer.

3. Mediterranean Stuffed Chicken Breasts - Mediterranean cuisine: This dish involves stuffing chicken breasts with chopped sun-dried tomatoes, black olives, feta cheese, and spinach, and serving it with a side salad or roasted vegetables.

4. Sun-Dried Tomato and Olive Pizza - Italia

In [15]:
# new input
sample_user_ingredient_input = "frozen shrimps"
messages = [
    {"role": "system", "content": "You are a recipe recommendation engine. Given a list of ingredients given by user, give at most 5 recipe that involves said ingredients. Return the name of the recipe, type of cuisine, and a brief description of the dish. If the dish is from a particular cuisine, also include the original language's name in English."},
    {"role": "user", "content": user_input_prefix_message + sample_user_ingredient_input}
]


In [16]:
chat_completion = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    temperature=temperature
)
response_text = chat_completion['choices'][0]['message']['content']

In [17]:
print(response_text)

Sure, here are five recipe recommendations that include frozen shrimps:

1. Shrimp Fried Rice (Chinese cuisine) - A classic Chinese dish made with shrimp, vegetables, and rice stir-fried together with soy sauce and other seasonings.

2. Creamy Garlic Shrimp Pasta (Italian cuisine) - A creamy and flavorful pasta dish made with shrimp, garlic, cream, Parmesan cheese, and pasta.

3. Shrimp Tacos (Mexican cuisine) - A tasty Mexican dish made with seasoned shrimp, tortillas, avocado, and other toppings like salsa and sour cream.

4. Shrimp and Grits (Southern cuisine) - A comforting Southern dish made with creamy grits and flavorful shrimp cooked with bacon, onions, and other seasonings.

5. Shrimp Curry (Indian cuisine) - A flavorful Indian dish made with shrimp, coconut milk, curry spices, and other seasonings, served with rice or naan.


In [18]:
# okay the output style looks...quite variable? Going to try to directly input the system message as user input
# as openai suggested 

sample_user_ingredient_input = "frozen shrimps"
instruction_text = "You are a recipe recommendation engine. Given a list of ingredients given by user, give at most 5 recipe that involves said ingredients. Return the name of the recipe, type of cuisine, and a brief description of the dish. If the dish is from a particular cuisine, also include the original language's name in English. \n"
messages = [
    {"role": "user", "content": instruction_text + user_input_prefix_message + sample_user_ingredient_input}
]


In [19]:
chat_completion = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    temperature=temperature
)
response_text = chat_completion['choices'][0]['message']['content']

In [20]:
print(response_text)

1. Garlic Butter Shrimp (American cuisine): This dish consists of juicy, succulent shrimp sautéed in garlic butter and served on a bed of rice. It's a simple yet elegant meal that can be prepared in less than 30 minutes.

2. Shrimp Fried Rice (Chinese cuisine): This classic Chinese dish is made with sautéed vegetables, rice, and scrambled eggs, all tossed together with tender shrimp. It's a delicious and filling meal that's perfect for a quick dinner.

3. Shrimp Scampi (Italian cuisine): This dish features succulent shrimp cooked in a buttery garlic sauce and served over a bed of pasta. It's a classic Italian dish that's easy to make and always a crowd-pleaser.

4. Shrimp Curry (Indian cuisine): This spicy and flavorful curry is made with tender shrimp cooked in a rich and creamy sauce made with coconut milk and a blend of aromatic spices. It's a delicious and satisfying meal that's perfect for a cozy night in.

5. Shrimp Tacos (Mexican cuisine): This twist on a classic Mexican dish fe

In [25]:
# okay trying with completions instead of chat. this is wild.
completion = openai.Completion.create(
    model='text-davinci-003',
    prompt=instruction_text + user_input_prefix_message + sample_user_ingredient_input,
    temperature=temperature,
)

In [8]:
# prompts are getting longer so checking how long the prompt is
# https://github.com/openai/tiktoken
import tiktoken
enc = tiktoken.encoding_for_model(model)

In [10]:
system_instruction = """
You are a recipe recommendation engine. Your task is to give at most 5 recipe that involves one or more ingredient. Return the name of the recipe, type of cuisine if it exists, and a brief description of the dish. If the dish is from a particular cuisine, also include the original language's name in English.

Example:
user input = "pork belly, anise"
Response:

1. Recipe: Chinese Braised Pork Belly (红烧肉 - Hóng Shāo Ròu)
Cuisine: Chinese
Description: Pork belly braised in sauce made of anise, soy sauce, ginger, and other aromatic spices. It is often served with steamed rice and vegetables.

2. Recipe: Spicy Anise Pork Belly Tacos
Cuisine: Mexican/Asian-inspired
Description: Taco includes marinated and grilled pork belly seasoned with spices, including anise, cumin, chili, and garlic. The pork belly is sliced and served in warm tortillas, along with your choice of toppings such as fresh cilantro, diced onions, and a squeeze of lime.
"""

len(enc.encode(system_instruction))

228

In [11]:
# measuring length of sample results got in original chat
example_response = """
Here are five recipe recommendations that involve pork belly and anise:

Recipe: Chinese Braised Pork Belly (红烧肉 - Hóng Shāo Ròu)
Cuisine: Chinese
Description: This classic Chinese dish, known as "Hong Shao Rou," features succulent pork belly braised in a flavorful sauce infused with anise, soy sauce, ginger, and other aromatic spices. The result is tender, melt-in-your-mouth pork belly with a rich, savory taste. It is often served with steamed rice and vegetables.

Recipe: Star Anise and Soy Glazed Pork Belly
Cuisine: Fusion/Asian-inspired
Description: This recipe combines the rich flavors of star anise and soy sauce to create a sweet and savory glaze for crispy pork belly. The pork belly is roasted until crispy on the outside and tender on the inside, and then glazed with a mixture of star anise, soy sauce, honey, and other ingredients. It's a delightful fusion of flavors.

Recipe: Slow-Cooked Pork Belly with Anise and Orange
Cuisine: Mediterranean
Description: This Mediterranean-inspired recipe features slow-cooked pork belly infused with the aromatic combination of anise and orange. The pork belly is braised with anise seeds, orange zest, garlic, and other seasonings, resulting in a tender and flavorful dish. It pairs well with roasted vegetables or a fresh salad.

Recipe: Spicy Anise Pork Belly Tacos
Cuisine: Mexican/Asian-inspired
Description: These flavorful tacos feature marinated and grilled pork belly seasoned with spices, including anise, cumin, chili, and garlic. The pork belly is sliced and served in warm tortillas, along with your choice of toppings such as fresh cilantro, diced onions, and a squeeze of lime. It's a delicious fusion of Mexican and Asian flavors.

Recipe: Anise-Glazed Pork Belly Skewers
Cuisine: Barbecue/Grilling
Description: These skewers are perfect for a barbecue or grilling session. Tender pieces of pork belly are marinated with a glaze made from anise, soy sauce, honey, and other ingredients, and then grilled to perfection. The result is a mouthwatering combination of smoky, sweet, and savory flavors.

Enjoy preparing these delectable dishes featuring pork belly and anise!
"""

len(enc.encode(example_response))


481

In [ ]:
# final design I'm going for right now

"""
You are a recipe recommendation engine. Your task is to give at most 5 recipes that involves one or more ingredient given by the user. Return the name of the recipe, type of cuisine if it exists, and a brief description of the dish. If the dish is from a particular cuisine, also include the original language name and english pronunciation of the name. If recipe is not from a specific cuisine, give back 'no specific cuisine' as cuisine type. Give result back in the following format:
- <recipe title> (<original language title>, english pronunciation)
- cuisine type
- description

ingredients: pork belly, anise
"""